In [2]:
import glob
import cv2
import numpy as np
from utils import *
import pandas as pd
from segmentation import breakLines, breakWords, extractTemplate, templateMatch, getBaseLine, addEmptySpaceSep, matchFirstCharacter

In [ ]:
def datasetToCsv(img_dir, text_dir, image_files, destination):
    rows = []

    column_names = ["feature" + str(i) for i in range(25*25)]
    column_names.append("label")
    count = 0
    for image in image_files:
        if limit > 0 and count == limit:
            break
        if count % 200 == 0:
            print("currently at image: " + str(count))
        img_path = img_dir + "/" + image
        text_path = text_dir + "/" + image.split('.')[0] + ".txt"

        img = cv2.imread(img_path)
        img_words = imageToWords(img)
        with open(text_path) as f:
            text = f.readlines()[0]
            text_words = text.split()
            index = 0
            for word in text_words:
                if index >= len(img_words):
                    break
                chars = segmenteCharacters(img_words[index])
                if len(chars) == len(word):
                    chars.reverse()
                    for i in range(len(word)):
                        letter = extractTemplate(chars[i])
                        if letter.shape[0] < 25 and letter.shape[1] < 25:
                            mask = np.zeros((25, 25))

                            vertical_start = int((25 - letter.shape[0]) / 2)
                            vertical_end = vertical_start + letter.shape[0]
                            horizontal_start = int((25 - letter.shape[1]) / 2)
                            horizontal_end = horizontal_start + letter.shape[1]

                            mask[vertical_start:vertical_end,
                                 horizontal_start:horizontal_end] = letter
                            mask = mask.reshape(1, mask.shape[0]*mask.shape[1])
                            row = [mask[:, i][0] for i in range(mask.shape[1])]
                            row.append(word[i])
                            rows.append(row)
                            LETTER_COUNTS[word[i]] += 1
                index += 1
        count += 1

    df = pd.DataFrame(data=rows, columns=column_names)
    df.to_csv(destination + "/dataset.csv", index=False)

In [12]:
def datasetToCsv(dataset_path):
    img_dirs = glob.glob(dataset_path+"/*")

    column_names = ["feature" + str(i) for i in range(25*25)]
    column_names.append("label")
    rows = []
    for img_dir in img_dirs:
        img_pathes = glob.glob(img_dir+"/*")
        print("currently at letter: " + img_dir.split('/')[-1])
        for img_path in img_pathes:
            img = (cv2.imread(img_path, 0) / 255).astype(np.uint8)
            img = img.reshape(1, img.shape[0]*img.shape[1])
            row = [img[:, i][0] for i in range(img.shape[1])]
            row.append(img_dir.split('/')[-1])
            rows.append(row)

    print("creating df...")
    df = pd.DataFrame(data=rows, columns=column_names)
    df.to_csv("./dataset.csv", index=False)

currently at letter: ع
currently at letter: ز
currently at letter: ج
currently at letter: ب
currently at letter: ن
currently at letter: ث
currently at letter: ر
currently at letter: ذ
currently at letter: ط
currently at letter: ف
currently at letter: ت
currently at letter: و
currently at letter: س
currently at letter: ﻻ
currently at letter: ل
currently at letter: غ
currently at letter: ا
currently at letter: د
currently at letter: ك
currently at letter: م
currently at letter: ظ
currently at letter: ح
currently at letter: خ
currently at letter: ه
currently at letter: ق
currently at letter: ش
currently at letter: ي
currently at letter: ض
currently at letter: ص
creating df...


In [15]:
img_dirs = glob.glob("./dataset/*")
labels = [img_dir.split('/')[-1] for img_dir in img_dirs]

df = pd.read_csv("./dataset.csv")
df = df.loc[:, (df != 0).any(axis=0)]

df = df.reset_index()

test_dataframes = []
for label in labels:
    test_dataframes.append(df.loc[df['label'] == label].sample(n=10))
    
test_df = pd.concat(test_dataframes)
train_df = df.drop(test_df.index)

train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)
del train_df['index']
del test_df['index']
train_df.to_csv("./train.csv", index=False)
test_df.to_csv("./test.csv", index=False)

In [20]:
features = [116, 117, 134, 135, 136, 137, 138, 139, 140, 141, 143, 145, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 301, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 482, 483, 484, 485, 486, 487, 488, 489, 490]


[116, 117, 134, 135, 136, 137, 138, 139, 140, 141, 143, 145, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 301, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 377, 378, 379, 380, 381, 382,